In [ ]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import hyspec_io, image_render, preprocess
# import matplotlib.colors
import tqdm
# import skimage.exposure

# import sklearn.metrics
# import spectral

In [ ]:
hyspec_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/2a_Rrs')
water_mask_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/M_Annotation_WaterMask/20220823_VegaSola_WaterMask_Gray')
rgb_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/2c_Rrs_RGB-WaterMask')
subfolders = ['Coast1','Coast2','Coast3','Sound1','Sound2','Sound3a','Sound3b']


In [ ]:
# Loop through all folders and images and create percentile stretched RGB image
for subfolder in subfolders:
    print(f'Processing subfolder {subfolder}')
    
    # Construct folder paths for current subfolder
    hyspec_folder = hyspec_base / subfolder
    water_mask_folder = water_mask_base / subfolder
    rgb_folder = rgb_base / subfolder
    
    # Create RGB subfolder if it doesn't exist
    rgb_folder.mkdir(exist_ok=True)

    # Loop through all images which have water masks (some or all of image has water)
    # Inpaint missing pixels and mask out non-water areas, save RGG images
    image_stems = [pathlib.Path(image_path).stem for image_path in sorted(water_mask_folder.glob('*.png'))]
    for image_stem in tqdm.tqdm(image_stems):
        (rgb_im,_,_,_) = hyspec_io.load_envi_image(str(hyspec_folder/image_stem)+'.hdr' , rgb_only=True)
        water_mask = np.expand_dims(skimage.io.imread(str(water_mask_folder/image_stem)+'.png'), 2)
        zeros_mask = np.all(rgb_im==0,axis=2,keepdims=True)
        rgb_im_inpainted = preprocess.inpaint_masked(rgb_im,np.squeeze(water_mask*zeros_mask))
        rgb_im_out = np.uint8(image_render.percentile_stretch(rgb_im_inpainted*water_mask,percentiles=(2,95))*255)
        skimage.io.imsave(str(rgb_folder/image_stem)+'.png',rgb_im_out, check_contrast=False)